# 외국인등록증 뒷면 분류기 훈련 노트북

이 노트북은 외국인등록증 뒷면을 분류하는 모델을 훈련하고 평가하는 과정을 담고 있습니다.

## 목표
- 이진 분류: 외국인등록증 뒷면 vs 기타 문서
- TensorFlow.js로 변환하여 웹에서 실시간 사용
- 경량화된 모델로 빠른 추론 속도 확보

In [ ]:
# 필요한 라이브러리 import
import sys
import os

# 프로젝트 루트를 경로에 추가
sys.path.append('../src')

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json

# 프로젝트 모듈 import
from data_utils import DataLoader, augment_data, visualize_samples
from model import create_mobilenet_classifier, create_efficient_classifier, get_model_summary

print(f"TensorFlow 버전: {tf.__version__}")
print(f"GPU 사용 가능: {len(tf.config.list_physical_devices('GPU'))}개")

# GPU 메모리 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

## 1. 데이터 준비 및 탐색

In [ ]:
# 데이터 경로 설정
DATA_DIR = '../data'
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# 데이터 로더 생성
data_loader = DataLoader(
    data_dir=DATA_DIR,
    img_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

# 데이터셋 통계 확인
train_path = os.path.join(DATA_DIR, 'train')
val_path = os.path.join(DATA_DIR, 'validation')

def count_files(directory):
    if not os.path.exists(directory):
        return 0
    return len([f for f in os.listdir(directory) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

# 훈련 데이터 통계
train_foreigner_count = count_files(os.path.join(train_path, 'foreigner_card_back'))
train_other_count = count_files(os.path.join(train_path, 'other_documents'))

# 검증 데이터 통계
val_foreigner_count = count_files(os.path.join(val_path, 'foreigner_card_back'))
val_other_count = count_files(os.path.join(val_path, 'other_documents'))

print("=== 데이터셋 통계 ===")
print(f"훈련 데이터:")
print(f"  - 외국인등록증 뒷면: {train_foreigner_count}장")
print(f"  - 기타 문서: {train_other_count}장")
print(f"  - 총합: {train_foreigner_count + train_other_count}장")
print(f"\n검증 데이터:")
print(f"  - 외국인등록증 뒷면: {val_foreigner_count}장")
print(f"  - 기타 문서: {val_other_count}장")
print(f"  - 총합: {val_foreigner_count + val_other_count}장")

# 클래스 비율 확인
if train_foreigner_count + train_other_count > 0:
    foreigner_ratio = train_foreigner_count / (train_foreigner_count + train_other_count)
    print(f"\n클래스 비율:")
    print(f"  - 외국인등록증 뒷면: {foreigner_ratio:.2%}")
    print(f"  - 기타 문서: {1-foreigner_ratio:.2%}")

## 10. 요약 및 다음 단계

In [ ]:
print("=== 프로젝트 완료 요약 ===")
print("\n✅ 완료된 작업:")
print("1. 📁 프로젝트 구조 생성")
print("2. 🤖 효율적인 CNN 모델 정의")
print("3. 📊 데이터 로더 및 전처리 파이프라인")
print("4. 🔄 데이터 증강 기능")
print("5. 📈 훈련 및 평가 코드")
print("6. 🌐 TensorFlow.js 변환 스크립트")
print("7. 💻 웹 데모 (HTML + JavaScript)")
print("8. 📓 Jupyter 노트북 가이드")

print("\n🔄 다음 단계:")
print("1. 📸 데이터 수집:")
print("   - 외국인등록증 뒷면 이미지 수집")
print("   - 기타 문서 이미지 수집 (신분증, 여권, 운전면허증 등)")
print("   - 다양한 각도, 조명, 배경으로 촬영")
print("\n2. 🏋️ 모델 훈련:")
print("   - python src/train_model.py 실행")
print("   - 또는 이 노트북의 훈련 셀 실행")
print("\n3. 🔄 TensorFlow.js 변환:")
print("   - python src/convert_to_tfjs.py --model_path models/best_model.h5")
print("\n4. 🌐 웹 데모 테스트:")
print("   - 로컬 서버에서 테스트")
print("   - 실제 이미지로 정확도 확인")
print("\n5. 🚀 프로덕션 배포:")
print("   - 웹 서버에 배포")
print("   - API 서버와 연동")
print("   - 모바일 최적화")

print("\n📋 권장 데이터 수량:")
print("- 외국인등록증 뒷면: 최소 500장, 권장 1000+장")
print("- 기타 문서: 최소 500장, 권장 1000+장")
print("- 훈련/검증 비율: 80/20 또는 70/30")

print("\n⚡ 성능 최적화 팁:")
print("- 이미지 품질: 고해상도보다는 다양성이 중요")
print("- 데이터 증강: 회전, 밝기, 대비 조정으로 견고성 향상")
print("- 모델 크기: TensorFlow.js용으로 2MB 이하 권장")
print("- 추론 속도: 모바일에서 100ms 이하 목표")

print("\n🔧 문제 해결:")
print("- 과적합: 더 많은 데이터, 강한 정규화")
print("- 저조한 성능: 더 복잡한 모델, 더 많은 데이터")
print("- 느린 추론: 모델 압축, 양자화 적용")
print("- 웹캠 권한: HTTPS 환경에서만 작동")